In [ ]:
!pip install -q -U bitsandbytes --no-index --find-links ../input/llama3-1-dependencies/dependencies/
!pip install -q -U transformers --no-index --find-links ../input/llama3-1-dependencies/dependencies/
!pip install -q -U tokenizers --no-index --find-links ../input/llama3-1-dependencies/dependencies/
!pip install -q -U peft --no-index --find-links ../input/llama3-1-dependencies/dependencies/

In [ ]:
!pip install -U trl

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import copy
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    DataCollatorWithPadding,
    LlamaForSequenceClassification,
    LlamaTokenizerFast,
    PreTrainedTokenizerBase,
    EvalPrediction,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType, PeftModel
from sklearn.metrics import log_loss, accuracy_score

In [ ]:
@dataclass
class Config:
    output_dir: str = "output"
    checkpoint: str = "/kaggle/input/unsloth-meta-llama-3.1-8b-bnb-4bit/transformers/default/1/Meta-Llama-3.1-8B-bnb-4bit"
    max_length: int = 2048
    n_splits: int = 5
    fold_idx: int = 0
    optim_type: str = "adamw_8bit"
    per_device_train_batch_size: int = 4
    gradient_accumulation_steps: int = 4
    per_device_eval_batch_size: int = 8
    n_epochs: int = 1
    freeze_layers: int = 16  # there're 32 layers in total, we don't add adapters to the first 16 layers
    lr: float = 2e-4
    warmup_steps: int = 20
    lora_r: int = 4
    lora_alpha: float = lora_r * 2
    lora_dropout: float = 0.05
    lora_bias: str = "none"

config = Config()

In [ ]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    target_modules=["q_proj", "k_proj", "v_proj"],
    layers_to_transform=[i for i in range(32) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
)

In [ ]:
tokenizer = LlamaTokenizerFast.from_pretrained(config.checkpoint)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True  # We'll add <eos> at the end
tokenizer.padding_side = "right"

In [ ]:
model = LlamaForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=3,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [ ]:
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model

In [ ]:
model.print_trainable_parameters()

In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
df = df.dropna()
df = df.drop_duplicates(subset=['response_a', 'response_b'], keep=False)
df["len"] = df["prompt"].apply(len) + df["response_a"].apply(len) + df["response_b"].apply(len)
df = df.sort_values(by=['len'])
df

In [ ]:
ds = Dataset.from_pandas(df)
ds = ds.select(torch.arange(1000)) #for demo purposes only

In [ ]:
class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __call__(self, batch: dict) -> dict:
        prompt = ["Which is the better response for the prompt? response_a or response_b or tie? \n'n give score for each lable \n\n <prompt>: " + self.process_text(t) for t in batch["prompt"]]
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)
        labels=[]
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):
            if a_win:
                label = 0
            elif b_win:
                label = 1
            else:
                label = 2
            labels.append(label)
        return {**tokenized, "labels": labels}
        
    @staticmethod
    def process_text(text: str) -> str:
        return " ".join(eval(text, {"null": ""}))

In [ ]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)
ds = ds.map(encode, batched=True)

In [ ]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()
    # Check for NaNs in predictions and labels
    if np.isnan(probs).any() or np.isnan(labels).any():
        raise ValueError("NaN values found in predictions or labels")

    loss = log_loss(y_true=labels, y_pred=probs)
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))
    return {"acc": acc, "log_loss": loss}

In [ ]:
folds = [
        (
            [i for i in range(len(ds)) if i % config.n_splits != fold_idx],
            [i for i in range(len(ds)) if i % config.n_splits == fold_idx]
        ) 
        for fold_idx in range(config.n_splits)
    ]

In [ ]:
from trl import SFTTrainer, SFTConfig
sft_config = SFTConfig(
    output_dir="output",
    overwrite_output_dir=True,
    report_to="none",
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=1000,
    save_strategy="epoch",
    save_steps=100,
    optim=config.optim_type,
    fp16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
    packing=True, 
    dataset_text_field="text",
    max_seq_length=config.max_length,
)

In [ ]:
trainer = SFTTrainer(
        model,
        train_dataset=ds,
        args=sft_config
    )

In [ ]:
for fold_idx in range(config.n_splits):
    
    train_idx, eval_idx = folds[fold_idx]

    train_data = ds.select(train_idx).sort("len")
    val_data = ds.select(eval_idx).sort("len")
    
    #split training data into batches with the same range of length
    batch_size = 200
    num_batches = len(train_data) // batch_size + (1 if len(train_data) % batch_size != 0 else 0)
    
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, len(train_data))
        ds_temp = train_data.select(range(start_idx, end_idx))
        
        trainer.train_dataset = ds_temp
        
        print(f"Training batch {batch_idx + 1}/{num_batches} on fold {fold_idx + 1}/{config.n_splits}...")
        
        trainer.train()
        
        trainer.save_model(f"model_fold_{fold_idx}_batch{batch_idx}")

    
    # Validate after training on all batches
    trainer.eval_dataset = val_data
    
    print(f"Validating on fold {fold_idx + 1}/{config.n_splits}...")
    eval_results = trainer.evaluate()

    # Save metrics if needed
    print(f"Evaluation results for fold {fold_idx + 1}: {eval_results}")